In [14]:
!pip install qdrant-client langchain langchain-qdrant langchain_community langchain_google_genai

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
cd /content/drive/MyDrive/gdg_solution_challenge

/content/drive/MyDrive/gdg_solution_challenge


In [17]:
!pip install google-cloud-translate

In [18]:
import os
from uuid import uuid4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableMap
from google.cloud import translate_v2 as translate
from qdrant_client.http.models import Distance, VectorParams, Filter, FieldCondition, MatchValue

In [20]:
# ✅ 환경 변수 설정
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["QDRANT_URL"] = QDRANT_URL
os.environ["QDRANT_API_KEY"] = QDRANT_API_KEY
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./credentials.json"

In [30]:
# ✅ GCP 번역 클라이언트 초기화
translate_client = translate.Client()

# ✅ 언어 감지 및 번역 관련 유틸
LANGUAGE_MAP = {"ko": "Korean", "ja": "Japanese", "zh": "Chinese", "en": "English"}
SUPPORTED_LANGUAGES = LANGUAGE_MAP.keys()

def detect_language(text: str) -> str:
    """입력 문장의 언어를 감지"""
    return translate_client.detect_language(text)["language"]


def translate_text(text: str, target_lang: str = "en") -> str:
    """

    텍스트를 target_lang으로 번역 (기본 영어)
    :param text: 원문 텍스트
    :param target_lang: "ko", "ja", "zh", "en" 중 선택
    :return: 원문, 번역문, 원문언어

    """

    # 이미 대상 언어인 경우 그대로 반환
    if detect_language(text) == target_lang:
        return text
    return translate_client.translate(text, target_language=target_lang)["translatedText"]

# ✅ 사용자 입력 준비 (영어 변환)
def prepare_input_in_english(user_inputs: dict) -> dict:
    source_lang = detect_language(user_inputs["symptoms"])
    user_inputs["source_lang"] = source_lang
    return {
        "image_analysis": translate_text(user_inputs["image_analysis"], "en"),
        "medical_history": translate_text(user_inputs["medical_history"], "en"),
        "symptoms": translate_text(user_inputs["symptoms"], "en"),
    }

# ✅ 결과를 target_lang으로 번역
def translate_output(output_text: str, target_lang: str) -> str:
    if detect_language(output_text) == target_lang:
        return output_text
    return translate_text(output_text, target_lang=target_lang)


In [22]:
# ✅ 폴더 안에 있는 텍스트 파일들을 모두 읽고 나누기
def load_and_split_documents(folder_path):
    documents = []

    # splitter 존재
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
                text = f.read()
                split_docs = splitter.create_documents([text])
                documents.extend(split_docs)
    return documents

# ✅ 문서 불러오기 및 임베딩
docs = load_and_split_documents("/content/drive/MyDrive/gdg_solution_challenge/llm_textDB")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


# ✅ Qdrant 업로드
vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_KEY"],
    collection_name="disease_collection",
    force_recreate=True,
)

# 검색기 생성 (상위 3개 문서 반환)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [41]:
# ✅ LLM 프롬프트 구성
template = """ You are a professional AI doctor specialized in dermatology.

Your task is to analyze patient information and return the top 3 most likely skin diseases with estimated probabilities.
Only choose from the following 12 diseases:
1. Atopic Dermatitis
2. Seborrheic Dermatitis
3. Impetigo
4. Ringworm
5. Chickenpox
6. Roseola (Exanthem subitum)
7. Contact Dermatitis
8. Miliaria
9. Milia
10. Erythema Toxicum Neonatorum
11. Aplasia Cutis Congenita
12. Measles
13. Noproblem

Do not suggest any disease that is not on the list above.
If there are no visible symptoms or medical concerns, return "1. Noproblem - XX%" with appropriate justification.


Here is the patient data:

[1. Image Analysis]
{image_analysis}

[2. Medical History]
{medical_history}

[3. Symptoms]
{symptoms}

Relevant context (disease information):
{context}

1. Disease Name - XX%
   - Reason: ...
2. Disease Name - XX%
   - Reason: ...
3. Disease Name - XX%
   - Reason: ...
"""

prompt = ChatPromptTemplate.from_template(template)

# Gemini Pro 모델 초기화
# gemini-1.5-pro or gemini-2.0-flash 中 답변을 통한 정성적 성능 비교 바람
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)


# ✅ 문서 -> 문자열 변환
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# ✅ 문맥 생성기
def get_context(x):
    docs = retriever.invoke(x["symptoms"])  # 검색 쿼리로 symptoms 사용
    return format_docs(docs)


# ✅ RAG 체인 구성
# 이미지분석, 과거이력, 증상
rag_chain = (
    RunnableMap({
        "context": get_context,
        "image_analysis": lambda x: x["image_analysis"],
        "medical_history": lambda x: x["medical_history"],
        "symptoms": lambda x: x["symptoms"],
    })

    | prompt
    | llm
)

# 전체 처리 체인
def run_diagnosis_chain(user_inputs):
    if user_inputs.get("target_lang") not in SUPPORTED_LANGUAGES:
        raise ValueError("지원되지 않는 언어입니다. ko/zh/ja/en 중 하나를 선택하세요.")

    # 1. 영어 기반 입력으로 변환
    inputs_en = prepare_input_in_english(user_inputs)

    # 2. RAG 실행 (영어)
    result_en = rag_chain.invoke(inputs_en)

    # 3. 결과를 target 언어로 번역
    return translate_output(result_en.content, user_inputs["target_lang"])

In [42]:
query_inputs = {
    "image_analysis": "A 부위: 좁쌀처럼 생긴 흰색 돌기, B 부위: 약한 홍반, C 부위: 모공 막힘",
    "medical_history": "과거 여드름 치료 경험. 지루성 피부염 가족력.",
    "symptoms": "볼과 이마에 최근 좁쌀이 생겼고 햇빛에 의해 악화됨. 간헐적 가려움증.",
    "target_lang": "ko"
}

response = run_diagnosis_chain(query_inputs)
print(response)

제공된 정보를 바탕으로 가장 발생 가능성이 높은 피부 질환에 대한 저의 평가는 다음과 같습니다. 1. **비립종 - 60%** - 원인: &quot;기장 씨앗처럼 보이는 흰색 융기&quot;라는 설명은 비립종을 강력하게 시사합니다. 최근 뺨과 이마에 나타난 화이트헤드는 이를 더욱 뒷받침합니다. 2. **지루성 피부염 - 30%** - 원인: 환자는 지루성 피부염 가족력이 있습니다. 가벼운 홍반과 막힌 모공, 그리고 간헐적인 가려움증은 지루성 피부염과 관련이 있을 수 있으며, 특히 환자의 여드름 치료력(지루성 피부염과 겹치는 경우가 있음)을 고려할 때 더욱 그렇습니다. 햇빛에 의한 악화 또한 한 가지 요인입니다. 3. **접촉성 피부염 - 10%** - 이유: 밀리아나 지루성 피부염보다는 발생 가능성이 낮지만, 홍반과 가려움증이 있는 경우 접촉성 피부염을 나타낼 가능성이 있습니다. 특히, 여드름 치료를 받은 적이 있는 경우 자극을 유발할 수 있는 제품을 사용했을 가능성이 있습니다.


In [43]:
query_inputs = {
    "image_analysis": "Part A: White bumps in the form of millet, Part B: Mild erythema, Part C: Pore clogging",
    "medical_history": "Past acne treatment experience. Family history of seborrheic dermatitis.",
    "symptoms": "Recently developed white millet on cheek and forehead, exacerbated by sun exposure. Intermittent itching.",
    "target_lang": "en"
}

response = run_diagnosis_chain(query_inputs)
print(response)

Here's my assessment of the patient's condition, based on the provided information:

1.  **Seborrheic Dermatitis - 45%**
    *   Reason: The patient presents with white bumps, mild erythema, pore clogging, and a family history of seborrheic dermatitis. The description of "white millet" on the face, exacerbated by sun exposure and intermittent itching, aligns with the symptoms of seborrheic dermatitis, particularly its presentation on the face.

2.  **Milia - 30%**
    *   Reason: The description of "white bumps in the form of millet" strongly suggests milia. Milia are small, white cysts that commonly appear on the face, particularly the cheeks and forehead.

3.  **Contact Dermatitis - 15%**
    *   Reason: The patient reports intermittent itching and recently developed white millet on the cheek and forehead, which could be caused by contact dermatitis. The past acne treatment experience may have sensitized the skin, making it more susceptible to irritants or allergens.


In [40]:
query_inputs = {
    "image_analysis": "피부 표면은 깨끗하고 염증, 발진, 돌기 없음",
    "medical_history": "피부 질환 이력 없음. 가족력 없음.",
    "symptoms": "피부 이상 무. 가려움, 통증, 발진 없음.",
    "target_lang": "ko"
}
response = run_diagnosis_chain(query_inputs)
print(response)

1. 문제 없음 - 99% - 이유: 이미지 분석, 병력 및 보고된 증상 모두 건강한 피부를 나타냅니다. 발진, 혹, 가려움증 또는 기타 피부 이상에 대한 언급은 없습니다. 환자에게 관련 증상이 나타나지 않으므로 제공된 신생아 중독성 홍반, 선천성 피부 무형성증 및 아토피 피부염에 대한 맥락은 무관합니다. 2. 비립종 - &lt;1% - 이유: 명확한 피부 설명을 고려하면 발생 가능성이 매우 낮지만, 비립종은 작고 흰색의 낭종으로, 건강한 피부에도 때때로 존재할 수 있습니다. 비립종은 대개 무증상이며 우려할 필요가 없습니다. 이미지 분석 결과 피부가 &quot;깨끗하다&quot;고 나타나므로 가능성은 희박합니다. 3. 신생아 중독성 홍반 - &lt;1% - 이유: 제공된 텍스트에는 신생아 중독성 홍반이 설명되어 있지만, 환자 데이터에는 발진이나 피부 이상이 없다고 명시되어 있습니다. 이 질환은 발진을 동반하므로, 제공된 정보를 고려할 때 발생 가능성은 매우 낮습니다. &quot;깨끗한 피부&quot; 평가에서 매우 미묘하고 초기 증상을 놓친 경우에만 가능성이 희박한 것으로 간주됩니다.


In [26]:
docs = retriever.invoke("백선에 대해서 알려줘?")
print(docs)  # 문서가 비어 있다면 문제 있음

[Document(metadata={'_id': '9a74127b-4d34-484e-be14-66c1d6169022', '_collection_name': 'disease_collection'}, page_content='"Miliaria" (Miliaria)'), Document(metadata={'_id': 'b044ceb3-c79c-4409-8a6a-e5f19e6d1751', '_collection_name': 'disease_collection'}, page_content='Language:\n\nEnglish'), Document(metadata={'_id': '465957ce-f5be-47e4-b1e8-495e992fbc1a', '_collection_name': 'disease_collection'}, page_content='6lb%3D%26pl%3D0%26plno%3D%26fi%3D0%26langcode%3Den%26upl%3D0%26cufr%3D%26cuto%3D%26howler%3D%26cvrem%3D0%26cvtype%3D0%26cvloc%3D0%26cl%3D0%26upfr%3D%26upto%3D%26primcat%3D%26seccat%3D%26cvcategory%3D*%26restriction%3D%26random%3D%26ispremium%3D1%26flip%3D0%26contributorqt%3D%26plgalleryno%3D%26plpublic%3D0%26viewaspublic%3D0%26isplcurate%3D0%26imageurl%3D%26saveQry%3D%26editorial%3D%26t%3D0%26apaid%3D%7B18B189B6-6A83-41BD-8442-2448A6B7E281%7D%26custspecid%3D14369B5F-24B7-4344-B743-D5DE569A1F46%26filters%3D0')]
